In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import tensorflow as tf
from tensorflow import keras
import os
import cv2

In [2]:
train_dir="C:/Users/kaval/Downloads/FaceMaskDetection-master/FaceMaskDetection-master/dataset"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset ='training'
)

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation')

Found 3619 images belonging to 2 classes.
Found 903 images belonging to 2 classes.


In [7]:
IMG_SIZE = 150
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [8]:
base_model.trainable = False
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 151, 151, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 75, 75, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 75, 75, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 75, 75, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 75, 75, 32) 

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [8]:
prediction_layer = keras.layers.Dense(1)

In [9]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 3,229,889
Trainable params: 1,025
Non-trainable params: 3,228,864
_________________________________________________________________


In [10]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
batch_size=32

In [14]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples // batch_size,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // batch_size)

'history = model.fit_generator(\n      train_generator,\n      steps_per_epoch=train_generator.samples // batch_size,\n      epochs=20,\n      validation_data=validation_generator,\n      validation_steps=validation_generator.samples // batch_size)'

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
model.save("pretrained_face2.h5")

In [2]:
from tensorflow.keras.models import load_model

In [3]:
model=load_model("pretrained_face2.h5")

In [4]:
 face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [6]:
source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        result=model.predict_classes(reshaped)
        print(result)

        #label=np.argmax(result,axis=1)[0]
        print(result[0][0])
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[result[0][0]],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[result[0][0]],-1)
        cv2.putText(img, labels_dict[result[0][0]], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[0]]
0
[[1]]
1
[[0]]
0
[[1]]
1
[[0]]
0
[[0]]
0
[[0]]
0
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[0]]
0
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1
[[1]]
1


In [ ]:
import datetime

In [ ]:
cap=cv2.VideoCapture(0)

while cap.isOpened():
    _,img=cap.read()
    face_img=cv2.resize(img,(150,150))
    face_img=face_img/255.0
    face_img=np.reshape(face_img,(150,150,3))
    face_img=np.expand_dims(face_img,axis=0)
    pred=model.predict_classes(face_img)
    if pred[0][0]==1:
        cv2.putText(img,'NO MASK',(250,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),4)
    else:
        cv2.putText(img,'MASK',(250,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4)
    datet=str(datetime.datetime.now())
    cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
        
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()